We modify Legendre's formula to produce Meissel's formula. Put $b = \pi(\sqrt{x})$ and $c = \pi(\sqrt[3]{x})$. Then
\begin{equation}
    \pi(x) = \psi(x, c) + \frac{1}{2}(b + c − 2)(b − c + 1) − \sum_{c < i \leq b} π\left( \frac{x}{p_i} \right).
\end{equation}
Note that it is now necessary to compute the values of $\pi(y)$ for certain values of $y$ in the range $x^{1/2}$ to $x^{2/3}$, and to have a list of primes up to $\sqrt{x}$. We save by only having to compute $\psi(x, \pi(\sqrt[3]{x}))$ rather than $\psi(x, \pi(\sqrt{x}))$.

In [3]:
import math

MAX_SQRT_X = 1000

def sieve(n):
    '''
    Generates a list of prime numbers up to n.
    '''
    primes_bool = [True] * (n + 1)
    if n >= 0: primes_bool[0] = False
    if n >= 1: primes_bool[1] = False
    for i in range(2, int(math.sqrt(n)) + 1):
        if primes_bool[i]:
            for multiple in range(i*i, n + 1, i):
                primes_bool[multiple] = False
    return [i for i, is_p in enumerate(primes_bool) if is_p]

PRIMES_CACHE = sieve(MAX_SQRT_X)

K = 6  # Optimization level for psi
FIRST_K_PRIMES = PRIMES_CACHE[:K]
M_K = 1
for p in FIRST_K_PRIMES: M_K *= p
PHI_M_K = M_K
for p in FIRST_K_PRIMES: PHI_M_K = PHI_M_K // p * (p - 1)
psi_k_table = list(range(M_K + 1))
for p in FIRST_K_PRIMES:
    for j in range(M_K, 0, -1):
        psi_k_table[j] -= psi_k_table[j // p]

psi_memo = {}
pi_memo = {}

def psi(x, a):
    '''
    Computes ψ(x, a) using the optimized recursion
    This function is used directly by the Meissel formula.
    '''
    x_int = int(x)
    if (x_int, a) in psi_memo:
        return psi_memo[(x_int, a)]
    if a == K:
        s = x_int // M_K
        t = x_int % M_K
        return s * PHI_M_K + psi_k_table[t]
    if a == 0:
        return x_int
    p_a = PRIMES_CACHE[a - 1]
    result = psi(x, a - 1) - psi(x / p_a, a - 1)
    psi_memo[(x_int, a)] = result
    return result

def pi_meissel(x):
    '''
    Computes π(x) using Meissel's formula.
    This function is recursive and uses memoization to store its own results.
    '''
    x = int(x)
    if x in pi_memo:
        return pi_memo[x]

    # Base case for recursion: for small values, use a direct count from the prime cache.
    if x <= MAX_SQRT_X:
        count = 0
        for p in PRIMES_CACHE:
            if p <= x: count += 1
            else: break
        pi_memo[x] = count
        return count

    # Calculate c = π(x^(1/3)) and b = π(√x) via recursive calls.
    c = pi_meissel(int(x**(1/3)))
    b = pi_meissel(int(math.sqrt(x)))

    term1 = psi(x, c)
    term2 = (b + c - 2) * (b - c + 1) // 2
    term3 = 0
    for i in range(c, b):
        p_i = PRIMES_CACHE[i]
        term3 += pi_meissel(x // p_i) # Recursive call
    result = term1 + term2 - term3
    pi_memo[x] = result
    return result

def run_tabulations():
    '''
    Clear caches and run the tabulation.
    '''
    global psi_memo, pi_memo
    for k in range(1, 6):
        r = range(10**k, 10**(k + 1) + 1, 10**k)
        title = f"x up to 10^{k + 1} (steps of 10^{k})"
        print(f"\nTable: {title}")
        print("-" * 26)
        print(f"{'x':>12} | {'π(x)':<12}")
        print("-" * 26)
        for x_val in r:
            psi_memo.clear()
            pi_memo.clear()
            pi_val = pi_meissel(x_val)
            print(f"{x_val:>12,} | {pi_val:<12,}")

run_tabulations()


Table: x up to 10^2 (steps of 10^1)
--------------------------
           x | π(x)        
--------------------------
          10 | 4           
          20 | 8           
          30 | 10          
          40 | 12          
          50 | 15          
          60 | 17          
          70 | 19          
          80 | 22          
          90 | 24          
         100 | 25          

Table: x up to 10^3 (steps of 10^2)
--------------------------
           x | π(x)        
--------------------------
         100 | 25          
         200 | 46          
         300 | 62          
         400 | 78          
         500 | 95          
         600 | 109         
         700 | 125         
         800 | 139         
         900 | 154         
       1,000 | 168         

Table: x up to 10^4 (steps of 10^3)
--------------------------
           x | π(x)        
--------------------------
       1,000 | 168         
       2,000 | 303         
       3,000 | 430         

The asymptotic behaviour of $\pi(x)$ is given by the prime number theorem
\begin{equation}
    \pi(x) \sim \frac{x}{\log{x}}
\end{equation}
and it is also known that a better approximation is
\begin{equation}
    \pi(x) \sim \mathop{Li}(x),
\end{equation}
where $\mathop{Li}$ is the logarithmic integral
\begin{equation}
    \mathop{Li}(x) = \int_x^0 \frac{dt}{\log t},
\end{equation}
suitably interpreted at the singular point $t = 1$. For computational purposes it is easier to take the lower limit of the integral to be $2$ and use the approximation $\mathop{Li}(2) \approx 1.045$.


In [4]:
import numpy as np

def pnt_approximation(x):
    '''
    Calculates the x / log(x) approximation.
    '''
    if x < 2: return 0
    return x / math.log(x)

def logarithmic_integral_li(x):
    '''
    Approximates the logarithmic integral Li(x) using Simpson's rule.
    '''
    if x < 2: return 0

    li2_approx = 1.045
    N = 10000  # Number of intervals for integration
    def f(t): return 1.0 / math.log(t)
    t_values = np.linspace(2, x, N + 1)
    y_values = np.array([f(t) for t in t_values])
    h = (x - 2) / N

    integral = h / 3.0 * (y_values[0] + 4 * np.sum(y_values[1:N:2]) + 2 * np.sum(y_values[2:N:2]) + y_values[N])
    return integral + li2_approx

def generate_tables_and_analysis():
    '''
    Generates the final tables comparing π(x) with its approximations.
    '''
    ranges = [
        range(100, 1001, 100),
        range(1000, 10001, 1000),
        range(10000, 100001, 10000),
        range(100000, 1000001, 100000)
    ]

    headers = ["x", "π(x)", "x/log(x)", "Li(x)", "π(x)/(x/logx)", "π(x)/Li(x)", "π(x)-x/logx", "π(x)-Li(x)"]

    for r in ranges:
        print("\n" + "-"*140)
        print(f"{headers[0]:>12} | {headers[1]:>12} | {headers[2]:>12} | {headers[3]:>12} | {headers[4]:>15} | {headers[5]:>12} | {headers[6]:>15} | {headers[7]:>15}")
        print("-" * 140)

        for x in r:
            psi_memo.clear()
            pi_memo.clear()

            # Calculate all required values
            pi_x_val = pi_meissel(x)
            approx_pnt = pnt_approximation(x)
            approx_li = logarithmic_integral_li(x)

            ratio_pnt = pi_x_val / approx_pnt if approx_pnt else 0
            ratio_li = pi_x_val / approx_li if approx_li else 0

            diff_pnt = pi_x_val - approx_pnt
            diff_li = pi_x_val - approx_li

            # Print the formatted row
            print(f"{x:>12,} | {pi_x_val:>12,} | {approx_pnt:>12,.2f} | {approx_li:>12,.2f} | {ratio_pnt:>15,.4f} | {ratio_li:>12,.4f} | {diff_pnt:>15,.2f} | {diff_li:>15,.2f}")

generate_tables_and_analysis()


--------------------------------------------------------------------------------------------------------------------------------------------
           x |         π(x) |     x/log(x) |        Li(x) |   π(x)/(x/logx) |   π(x)/Li(x) |     π(x)-x/logx |      π(x)-Li(x)
--------------------------------------------------------------------------------------------------------------------------------------------
         100 |           25 |        21.71 |        30.13 |          1.1513 |       0.8298 |            3.29 |           -5.13
         200 |           46 |        37.75 |        50.19 |          1.2186 |       0.9165 |            8.25 |           -4.19
         300 |           62 |        52.60 |        68.33 |          1.1788 |       0.9073 |            9.40 |           -6.33
         400 |           78 |        66.76 |        85.42 |          1.1683 |       0.9132 |           11.24 |           -7.42
         500 |           95 |        80.46 |       101.79 |          1.1808 |     

Relative error:
*   The ratio $\pi(x) / (x/\log{x})$ slowly decreases, approaching 1 from above. This shows that $x/\log(x)$ consistently underestimates the true number of primes, but the relative error improves as $x$ gets larger.
*   The ratio $\pi(x) / \mathop{Li}(x)$ slowly increases, approaching 1 from below. This shows that $\mathop{Li}(x)$ consistently overestimates the number of primes in this range.
*   By comparing the two ratios, it is evident that $\pi(x) / \mathop{Li}(x)$ is closer to $1$ than $\pi(x) / (x/\log(x))$ for any given $x$. This means $\mathop{Li}(x)$ has a smaller relative error.

Absolute error:
*   The difference $\pi(x) - x/\log(x)$ is always positive and grows steadily.
*   The difference $\pi(x) - \mathop{Li}(x)$ is always negative in this range. The absolute value also grows, but much more slowly

Both approximations demonstrate the asymptotic behavior described by the prime number theorem. Our program confirms the well-established fact that the logarithmic integral, $\mathop{Li}(x)$, provides a and more accurate approximation for the prime-counting function $\pi(x)$ than the simpler $x / \log(x)$ formula.


We analysing the integral for $\mathop{Li(x)}$ by making a change of variable. Up to a constant,
\begin{equation}
    \mathop{Li}(x) = \int_2^x \frac{1}{\log t} \,dt.
\end{equation}

Using the substitution $u = \log t$, we have $t = e^u$, so differentiating gives $dt = e^u du$. Substituting these into the integral gives
\begin{equation}
    \mathop{Li}(x) = \int_{\log 2}^{\log x} \frac{1}{u} e^u \,du.
\end{equation}
We can find an asymptotic expansion for it using repeated integration by parts. A first application gives
\begin{equation}
    \int \frac{e^u}{u} \,du = \frac{e^u}{u} + \int \frac{e^u}{u^2} \,du
\end{equation}

A second applications gives
\begin{equation}
    \int \frac{e^u}{u^2} \,du = \frac{e^u}{u^2} + 2\int \frac{e^2}{u^2} \,du
\end{equation}
Substituting this back, we get an expansion
\begin{equation}
    \mathop{Li}(x) = \frac{e^u}{u} + \frac{e^u}{u^2} + 2\int_{\log 2}^{\log x} \frac{e^u}{u^3} \,du
\end{equation}

Continuing this process yields an asymptotic series. For large $x$, the contribution from the lower limit log 2 is a constant and can be ignored. Evaluating at the upper limit $u = \log x$,
\begin{equation}
    \mathop{Li(x)} \approx \sum_{n=0}^{\infty} \frac{x}{(n)!(\log x)^{n+1}}.
\end{equation}

The prime number theorem states that $\pi(x) ≈ x/\log x$. Our expansion shows that $\mathop{Li(x)}$ agrees with this to first order, demonstrating why $\mathop{Li(x)}$ is a better approximation for $\pi(x)$.

The dominant term in the difference between the two common approximations is given by the quadratic term
\begin{equation}
    \mathop{Li}(x) - \frac{x}{\log x} \approx \frac{x}{(\log x)^2}.
\end{equation}

Since $\mathop{Li}(x)$ already accounts for all the smooth correction terms of the form $x/(\log x)^n$, the remaining error $\pi(x) - \mathop{Li}(x)$ must be of a different stochastic nature. In many random processes, the expected deviation from the mean after $x$ steps is related to $\sqrt{x}$.

It turns out that if the Riemann hypothesis is true, then the order of magnitude for the error term is
\begin{equation}
    \pi(x) - \mathop{Li}(x) = O(\sqrt{x}\log x).
\end{equation}